In [ ]:
#punya hans
import pandas as pd
from bs4 import BeautifulSoup
import re

# --- 1. FUNGSI UNTUK MENGAMBIL DATA DARI SATU HALAMAN HTML ---
def parse_laptop_data(html_content):
    """
    Mengurai data laptop dari konten HTML yang diberikan (halaman tunggal).
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    data_laptop = []

    # Cari semua elemen 'li' yang berisi data laptop.
    # Elemen ini adalah pembungkus untuk setiap produk laptop.
    laptop_listings = soup.find_all('li', class_='flex items-center gap-2 list-none border-b py-1 mb-2')

    for listing in laptop_listings:
        # Awalnya, cari tag <a> yang berisi nama dan detail laptop
        main_link = listing.find('a', class_=re.compile(r"grid-cols-laptopLayoutSmall"))
        if not main_link:
            continue
        
        # 1. Nama Laptop (berada di tag <h2> di dalam <a>)
        name_el = main_link.find('h2')
        name = name_el.get_text(strip=True) if name_el else 'N/A'
        
        # 2. Detail Spesifikasi (berada di tag <dl> di dalam <a>)
        specs = {}
        specs_container = main_link.find('dl')
        if specs_container:
            # Menggunakan dt (deskripsi term) dan dd (deskripsi detail) untuk mengekstrak spesifikasi
            dt_elements = specs_container.find_all('dt')
            dd_elements = specs_container.find_all('dd')
            
            # Map the specification details based on the <dt> tag content
            for dt, dd in zip(dt_elements, dd_elements):
                key = dt.get_text(strip=True).replace(':', '') 
                value = dd.get_text(strip=True)
                specs[key] = value

        # 3. Harga Jual
        price_el = listing.find('div', class_='priceBtn').find('span', class_='text-lm-darkBlue')
        price_raw = price_el.get_text(strip=True) if price_el else 'N/A'
        price_cleaned = re.sub(r'[$,€]', '', price_raw).replace(' ', '').replace('sup', '') # Membersihkan simbol mata uang dan spasi

        # 4. Link ke Halaman Detail Produk
        detail_url = main_link['href'] if main_link and 'href' in main_link.attrs else 'N/A'

        # 5. Link Beli (ke Amazon)
        buy_link_el = listing.find('a', target='_blank', href=re.compile(r'amazon\.com'))
        buy_link = buy_link_el['href'] if buy_link_el else 'N/A'
        
        data_laptop.append({
            'Nama_Laptop': name,
            'Processor': specs.get('Processor', 'N/A'),
            'RAM': specs.get('Internal memory', 'N/A'),
            'GPU': specs.get('Video card', 'N/A'),
            'Display': specs.get('Display', 'N/A'),
            # Menggabungkan SSD dan HDD karena keduanya menggunakan class 'storage'
            'Storage': specs.get('Solid-state drive', '') + ' ' + specs.get('Hard drive', ''),
            'Harga_USD': price_cleaned,
            'Detail_URL': detail_url,
            'Buy_Link': buy_link,
        })

    return data_laptop

In [ ]:
# Menerapkan dan menjalankan fungsi parsing
import pandas as pd
from bs4 import BeautifulSoup
import re

# (Definition of parse_laptop_data function is placed here before running)

# Membaca konten dari file 'halaman1.html'
file_name = 'halaman1.html'
with open(file_name, 'r', encoding='utf-8') as f:
    html_content = f.read()

data_final = parse_laptop_data(html_content)

# Menyimpan data ke DataFrame dan CSV
if data_final:
    df = pd.DataFrame(data_final)
    # Membersihkan kolom Storage yang mungkin memiliki spasi ganda dari penggabungan
    df['Storage'] = df['Storage'].str.strip().replace(r'\s+', ' ', regex=True)
    
    output_file = 'laptop_data_scraped.csv'
    df.to_csv(output_file, index=False)
    
    print("\n================ EKSTRAKSI DATA DARI HALAMAN 1 SUKSES ================")
    print(f"Data berhasil disimpan ke {output_file}")
    print(f"Total Data Laptop yang Diekstrak: {len(df)}")
    print("\nBerikut 5 baris pertama dari data yang diekstrak:")
    print(df.head())
else:
    print("Tidak ada data laptop yang ditemukan.")

# Output the dataframe for review
print(df.head())


================ EKSTRAKSI DATA DARI HALAMAN 1 SUKSES ================
Data berhasil disimpan ke laptop_data_scraped.csv
Total Data Laptop yang Diekstrak: 1000

Berikut 5 baris pertama dari data yang diekstrak:
                 Nama_Laptop                Processor       RAM  \
0        Alienware 16 Aurora        Intel Core 7 240H  24GB RAM   
1  Lenovo ThinkBook 16 Gen 8        Intel Core 7 240H  16GB RAM   
2           Lenovo V15 Gen 4        AMD Ryzen 7 7730U  12GB RAM   
3  Lenovo ThinkPad T16 Gen 3  Intel Core Ultra 7 155H  16GB RAM   
4   Samsung Galaxy Book4 360        Intel Core 7 150U  16GB RAM   

                                      GPU  \
0        NVIDIA GeForce RTX 5050 (Laptop)   
1   Intel UHD Graphics (Alder Lake, 64EU)   
2  AMD Radeon RX Vega 8 (R4000/5000, 15W)   
3                     Intel Arc (8-Cores)   
4        Intel Iris Xe Graphics G7 (96EU)   

                                   Display     Storage Harga_USD  \
0  16.0", WQXGA (2560 x 1600), 120 Hz, IPS  10

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re

def parse_laptop_data(html_content):
    """
    Mengurai data laptop dari konten HTML yang diberikan (halaman tunggal).
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    data_laptop = []

    # Target utama adalah setiap item <li> yang berisi data laptop
    # Class: flex items-center gap-2 list-none border-b py-1 mb-2
    #
    laptop_listings = soup.find_all('li', class_=re.compile(r"flex items-center.*border-b"))

    for listing in laptop_listings:
        # Cari tag <a> di dalam <li>, yang berisi detail spesifikasi dan link
        main_link = listing.find('a', class_=re.compile(r"grid-cols-laptopLayoutSmall"))
        if not main_link:
            continue
        
        # 1. Nama Laptop (berada di tag <h2>)
        name_el = main_link.find('h2')
        name = name_el.get_text(strip=True) if name_el else 'N/A'
        
        # 2. Detail Spesifikasi (berada di tag <dl>)
        specs = {}
        specs_container = main_link.find('dl')
        if specs_container:
            # Mengambil semua <dt> dan <dd> di dalam <dl>
            dt_elements = specs_container.find_all('dt')
            dd_elements = specs_container.find_all('dd')
            
            for dt, dd in zip(dt_elements, dd_elements):
                # Membersihkan nama spesifikasi (misalnya 'Processor' dari '<dt class="sr-only">Processor</dt>')
                key = dt.get_text(strip=True).replace(':', '') 
                value = dd.get_text(strip=True)
                specs[key] = value

        # 3. Harga Jual dan Link Beli
        price_btn = listing.find('div', class_='priceBtn')
        price_el = price_btn.find('span', class_='text-lm-darkBlue') if price_btn else None
        
        # Membersihkan harga dari simbol mata uang ($) dan tag <sup>
        price_raw = price_el.get_text(strip=True) if price_el else 'N/A'
        price_cleaned = re.sub(r'[$,€\s]', '', price_raw).split('sup')[0] 
        
        # Link ke Halaman Detail Produk
        detail_url = main_link['href'] if main_link and 'href' in main_link.attrs else 'N/A'

        # Link Beli (ke Amazon)
        buy_link_el = listing.find('a', target='_blank', href=re.compile(r'amazon\.com'))
        buy_link = buy_link_el['href'] if buy_link_el else 'N/A'
        
        # Menggabungkan semua data
        data_laptop.append({
            'Nama_Laptop': name,
            'Harga_USD': price_cleaned,
            'Processor': specs.get('Processor', 'N/A'),
            'RAM': specs.get('Internal memory', 'N/A'),
            'GPU': specs.get('Video card', 'N/A'),
            'Display': specs.get('Display', 'N/A'),
            # Menggabungkan SSD dan HDD karena keduanya menggunakan class 'storage'
            'Storage': f"{specs.get('Solid-state drive', '')} {specs.get('Hard drive', '')}".strip(),
            'Detail_URL': detail_url,
            'Buy_Link': buy_link,
        })

    return data_laptop

# --- Eksekusi pada file lokal ---
# Membaca konten dari file 'halaman1.html'
file_name = 'halaman1.html'
with open(file_name, 'r', encoding='utf-8') as f:
    html_content = f.read()

data_halaman1 = parse_laptop_data(html_content)
df_halaman1 = pd.DataFrame(data_halaman1)

print("\n================ EKSTRAKSI DATA DARI HALAMAN 1 SUKSES ================")
print(f"Total Data Laptop yang Diekstrak: {len(df_halaman1)}")
print("\nBerikut 5 baris pertama dari data yang diekstrak:")
print(df_halaman1.head())


================ EKSTRAKSI DATA DARI HALAMAN 1 SUKSES ================
Total Data Laptop yang Diekstrak: 1000

Berikut 5 baris pertama dari data yang diekstrak:
                 Nama_Laptop Harga_USD                Processor       RAM  \
0        Alienware 16 Aurora    124900        Intel Core 7 240H  24GB RAM   
1  Lenovo ThinkBook 16 Gen 8    127900        Intel Core 7 240H  16GB RAM   
2           Lenovo V15 Gen 4     56900        AMD Ryzen 7 7730U  12GB RAM   
3  Lenovo ThinkPad T16 Gen 3    119900  Intel Core Ultra 7 155H  16GB RAM   
4   Samsung Galaxy Book4 360    106099        Intel Core 7 150U  16GB RAM   

                                      GPU  \
0        NVIDIA GeForce RTX 5050 (Laptop)   
1   Intel UHD Graphics (Alder Lake, 64EU)   
2  AMD Radeon RX Vega 8 (R4000/5000, 15W)   
3                     Intel Arc (8-Cores)   
4        Intel Iris Xe Graphics G7 (96EU)   

                                   Display     Storage  \
0  16.0", WQXGA (2560 x 1600), 120 Hz, IPS  10

In [4]:
import undetected_chromedriver as uc
import time 

def scrape_multi_page(base_url, total_pages):
    all_laptop_data = []
    
    # Inisialisasi driver (undetected_chromedriver akan otomatis mendownload driver)
    # use_subprocess=False dan headless=True adalah opsi yang umum digunakan
    # Anda mungkin perlu menguji opsi ini. Hapus 'headless=True' jika ingin melihat browser.
    driver = uc.Chrome(use_subprocess=False, headless=True)
    
    # Asumsi: Halaman 1 sudah kita scrap, kita mulai dari halaman 2
    for page_num in range(1, total_pages + 1):
        # Pola URL di situs ini biasanya seperti: https://laptopmedia.com/specs/page/2/?size=n_1000_n
        url_halaman = f"https://laptopmedia.com/specs/page/{page_num}/?size=n_1000_n"
        print(f"Mengunjungi halaman: {url_halaman}")
        
        try:
            driver.get(url_halaman)
            # Beri jeda waktu agar halaman memuat sempurna dan melewati anti-bot (jika ada)
            time.sleep(5) 
            
            # Ambil konten HTML yang sudah di-render oleh browser
            html_content = driver.page_source
            
            # Panggil fungsi parsing yang sudah kita buat sebelumnya
            data_halaman = parse_laptop_data(html_content)
            
            # Hentikan perulangan jika tidak ada data yang ditemukan (mencapai akhir halaman)
            if not data_halaman:
                print("Tidak ada data baru ditemukan. Menghentikan scraping.")
                break
                
            all_laptop_data.extend(data_halaman)
            
        except Exception as e:
            print(f"Terjadi error saat scraping halaman {page_num}: {e}")
            break

    driver.quit() # Sangat penting untuk menutup driver setelah selesai
    return all_laptop_data

# --- Penggunaan ---
# Ganti angka 5 dengan jumlah halaman yang ingin Anda ambil (atau angka yang sangat besar untuk semua halaman)
# total_pages_to_scrape = 5 
# final_data = scrape_multi_page(base_url="https://laptopmedia.com/specs/", total_pages=total_pages_to_scrape)

# df_final = pd.DataFrame(final_data)
# df_final['Storage'] = df_final['Storage'].str.strip().replace(r'\s+', ' ', regex=True)
# df_final.to_csv('all_laptop_data.csv', index=False)
# print(f"\nScraping selesai. Total data yang diekstrak: {len(df_final)}")